In [110]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [280]:
df = pd.DataFrame(pd.read_csv('normalized_data.csv'))
df_input = df.drop(['target'],axis = 1)
df1 = pd.DataFrame(pd.read_csv('Audio_features.csv'))
df_new = df1.drop(['Id'],axis = 1)
df_new = pd.get_dummies(df_new)
target = df_new[['Gender_f','Gender_m']]
print(target.head())
print(df_input.head())

   Gender_f  Gender_m
0         1         0
1         1         0
2         1         0
3         1         0
4         1         0
   Max_pitch  Avg_pitch  harmonic  percussive    chroma     mfccs  contrast  \
0   0.134618   1.497048  0.013215   -0.025919 -0.435296 -0.275369  0.536295   
1   0.010203   0.015233  0.015147   -0.055289 -1.464748 -0.052996  0.727536   
2   0.146101   1.471575  0.015656   -0.028862 -0.977221 -0.088292  0.147186   
3   0.165802   1.705779  0.018674    0.034075  0.432367 -0.046729  0.374442   
4   0.119065   0.709232  0.014741   -0.054663 -0.036201  0.079671  0.596260   

    rolloff     zrate  Spectralcentroid  
0  1.555512  0.871418          1.152535  
1  1.029680 -0.268728          0.351913  
2  1.243102  0.716065          1.041939  
3  1.520395  0.782194          1.252516  
4  0.744706  0.467126          0.484467  


In [73]:
def relu(d):
    return (max(0,d))

In [195]:
import math 
def softmax(num):
    opt = np.array([])
    expnum = np.array([])
    s = 0
    for i in range(len(num)):
        expnum = np.append(expnum,math.exp(num[i]))
        s += math.exp(num[i])
        #print(expnum)
    for j in range(len(expnum)):
        g = expnum[j]/s
        opt = np.append(opt,round(g,3))
    return (opt)    

In [308]:
def NN_pred(input_data,weight):
    node_0_val = ((input_data*weight['node_0']).sum())
    node_1_val = ((input_data*weight['node_1']).sum())
    node_2_val = ((input_data*weight['node_2']).sum())
    hidden_layer = np.array([node_0_val,node_1_val,node_2_val])
    op_node0_val = ((hidden_layer*weight['op_node_0']).sum())
    op_node1_val = ((hidden_layer*weight['op_node_1']).sum())
    output_pred = softmax(np.array([op_node0_val,op_node1_val]))
    return(output_pred)

In [309]:
x1 =[]
def grad_desc(x,w,y):
    l_rate = 0.01
    mse = []
    for i in range(1,6):
        x1.append(i)
        
        node_0_val = ((x*w['node_0']).sum())
        node_1_val = ((x*w['node_1']).sum())
        node_2_val = ((x*w['node_2']).sum())
        hidden_layer = np.array([node_0_val,node_1_val,node_2_val])
        op_node0_val = ((hidden_layer*w['op_node_0']).sum())
        op_node1_val = ((hidden_layer*w['op_node_1']).sum())
        y_pred = softmax(np.array([op_node0_val,op_node1_val]))
        
        #y_pred = NN_pred(x,w)
        mse.append(np.sqrt((y[0]-y_pred[0])*(y[0]-y_pred[0])))
        for j in range(0,3):
            for k in range(0,len(w['node_'+str(j)])):
                dmdw = 2*(y[0]-y_pred[0])*(-x[k])                 
                w['node_'+str(j)][k] -= (l_rate*dmdw)
   
       
        node_0_val = ((x*w['node_0']).sum())
        node_1_val = ((x*w['node_1']).sum())
        node_2_val = ((x*w['node_2']).sum())
        hidden_layer = np.array([node_0_val,node_1_val,node_2_val])  
        #print('Weight 0 and 1:',w['node_0'],w['node_1'])
        #print('HiddenLayer :',hidden_layer)
        for l in range(0,2):
            for k in range(0,len(w['op_node_0'])):
                dmdw1 = 2*(y[l]-y_pred[l])*(-hidden_layer[k])
                #print('op_node_'+str(l)+' rate: ',dmdw1)
                #print('op_node_'+str(l)+'rate : {} ,Hidden_Layer'+str(k)+' : {}'.format(dmdw1,hidden_layer[k]))
                w['op_node_'+str(l)][k] -= (l_rate*dmdw1)
                #w['op_node_'+str(l)][k] = w2            
        #print(w1,w2)
        print('Y_Predicted: {} ,Weights {}'.format(y_pred,w['op_node_0']))
    print('RMSE : {}'.format(mse))
    return(mse)

In [312]:
input_data = np.array(df_input.loc[158])
weight = {'node_0': np.array([3,4,6,8,7,2,6,3,2,1]), 'node_1':np.array([0.1,0.3,0.4,0.8,0.3,1.2,3,5,2.6,0.1]),
         'node_2': np.array([0.5,0.3,0.7,0.8,4,3,7,2,9,1]), 'op_node_0': np.array([0.5,1.2,0.2]),'op_node_1': np.array([0.1,2,0.3])}
output = np.array(target.loc[158])
weight_copy = {'node_0': np.array([3,4,6,8,7,2,6,3,2,1]), 'node_1':np.array([0.1,0.3,0.4,0.8,0.3,1.2,3,5,2.6,0.1]),
         'node_2': np.array([0.5,0.3,0.7,0.8,4,3,7,2,9,1]), 'op_node_0': np.array([0.5,1.2,0.2]),'op_node_1': np.array([0.1,2,0.3])}

In [313]:
rmse = list()
rmse = grad_desc(input_data,weight,output)

Y_Predicted: [0.359 0.641] ,Weights [0.59220958 1.23022973 0.25938841]
Y_Predicted: [0.01 0.99] ,Weights [0.59496783 1.2310719  0.2610428 ]
Y_Predicted: [0.005 0.995] ,Weights [0.59635924 1.23149301 0.26187003]
Y_Predicted: [0.005 0.995] ,Weights [0.59775376 1.23191416 0.26269728]
Y_Predicted: [0.004 0.996] ,Weights [0.59887187 1.23225109 0.26335911]
RMSE : [0.359, 0.01, 0.005, 0.005, 0.004]


In [296]:
imp = np.array(df_input.loc[68])
NN_pred(imp,weight)

array([0.001, 0.999])

In [278]:
df = pd.DataFrame(pd.read_csv('Audio_features.csv'))
df_new = df.drop(['Id'],axis = 1)
df_new = pd.get_dummies(df_new)
target = df_new[['Gender_f','Gender_m']]
target.head()

,Gender_f,Gender_m
0,1,0
1,1,0
2,1,0
3,1,0
4,1,0


In [286]:
target.loc[159]

Gender_f    0
Gender_m    1
Name: 159, dtype: uint8